In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install jiwer
!pip install einops addict easydict

In [2]:
from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

README-checkpoint.md: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

processor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

'/content/deepseek_ocr'

In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from transformers import AutoModel
import os
os.environ["UNSLOTH_WARN_UNINITIALIZED"] = '0'
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Qwen3-VL-8B-Instruct-bnb-4bit", # Qwen 3 vision support
    "unsloth/Qwen3-VL-8B-Thinking-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Instruct-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Thinking-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "./deepseek_ocr",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    auto_model = AutoModel,
    trust_remote_code=True,
    unsloth_force_compile=True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.8: Fast Deepseekocr patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at ./deepseek_ocr and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
  # 1. Mount Drive để lấy file zip
  from google.colab import drive
  drive.mount('/content/drive')

  # 2. Copy file zip từ Drive vào ổ cứng nội bộ của Colab (/content/)
  # Giả sử bạn để file zip ở thư mục gốc của Drive
  !cp /content/drive/MyDrive/UIT_HWDB_line.zip /content/

  # 3. Giải nén
  # -q là quiet (không hiện danh sách file dài dằng dặc)
  !unzip -q /content/UIT_HWDB_line.zip -d /content/

  # 4. Kiểm tra xem folder đã xuất hiện chưa
  !ls /content/UIT_HWDB_line/train_data

Mounted at /content/drive
1    112  126  14   153  167  180  194	207  220  234  248  37	50  64	78  91
10   113  127  140  154  168  181  195	208  221  235  249  38	51  65	79  92
100  114  128  141  155  169  182  196	209  222  236  25   39	52  66	8   93
101  115  129  142  156  17   183  197	21   223  237  26   4	53  67	80  94
102  116  13   143  157  170  184  198	210  224  238  27   40	54  68	81  95
103  117  130  144  158  171  185  199	211  225  239  28   41	55  69	82  96
104  118  131  145  159  172  186  2	212  226  24   29   42	56  7	83  97
105  119  132  146  16	 173  187  20	213  227  240  3    43	57  70	84  98
106  12   133  147  160  174  188  200	214  228  241  30   44	58  71	85  99
107  120  134  148  161  175  189  201	215  229  242  31   45	59  72	86
108  121  135  149  162  176  19   202	216  23   243  32   46	6   73	87
109  122  136  15   163  177  190  203	217  230  244  33   47	60  74	88
11   123  137  150  164  178  191  204	218  231  245  34   48	61  75	89
110  124

In [6]:
import os
import json
from datasets import Dataset, Image as ImageD, DatasetDict

# Hàm chuẩn bị dữ liệu (giữ nguyên logic cũ)
def prepare_data(data_dir):
    data_list = []
    if not os.path.exists(data_dir):
        return None

    for folder_name in sorted(os.listdir(data_dir)):
        folder_path = os.path.join(data_dir, folder_name)
        if os.path.isdir(folder_path):
            json_path = os.path.join(folder_path, "label.json")
            if os.path.exists(json_path):
                with open(json_path, 'r', encoding='utf-8') as f:
                    labels = json.load(f)
                for img_name, text in labels.items():
                    img_path = os.path.join(folder_path, img_name)
                    if os.path.exists(img_path):
                        data_list.append({"image": img_path, "text": text})
    return data_list

# --- BẮT ĐẦU LOAD ---

# 1. Đường dẫn gốc
root_path = '/content/UIT_HWDB_line'

# 2. Tạo Dataset cho từng phần
train_list = prepare_data(os.path.join(root_path, 'train_data'))
test_list = prepare_data(os.path.join(root_path, 'test_data'))

# 3. Gộp vào DatasetDict
raw_datasets = {}
if train_list:
    raw_datasets["train"] = Dataset.from_list(train_list).cast_column("image", ImageD())
if test_list:
    raw_datasets["test"] = Dataset.from_list(test_list).cast_column("image", ImageD())



dataset = DatasetDict(raw_datasets)
# 1. Lấy tập train hiện tại ra để chia
train_val_split = dataset["train"].train_test_split(test_size=0.1, seed=3407)

# 2. Cập nhật lại DatasetDict để có đủ 3 tập: train, validation, test
dataset = DatasetDict({
    "train": train_val_split["train"],
    "valid": train_val_split["test"], # 10% trích từ train
    "test": dataset["test"]                # Tập test gốc giữ nguyên
})
#3: Kiểm tra kết quả
print(dataset)



DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 6325
    })
    valid: Dataset({
        features: ['image', 'text'],
        num_rows: 703
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 201
    })
})


In [7]:
import io
import numpy as np
import os
from contextlib import redirect_stdout
from jiwer import cer
from tqdm import tqdm

# --- 1. KIỂM TRA VÀ TẠO THƯ MỤC KẾT QUẢ ---
output_dir = "/content/drive/MyDrive/result" # Dùng đường dẫn tuyệt đối cho chắc chắn
os.makedirs(output_dir, exist_ok=True)

# --- 2. CẤU HÌNH TẬP TEST ---
num_samples = 201
# Đảm bảo bạn đang dùng đúng split 'test'
test_subset = dataset['test'].select(range(min(num_samples, len(dataset['test']))))
results = []

print(f"Bắt đầu đánh giá {len(test_subset)} mẫu...")

for i, example in enumerate(tqdm(test_subset)):
    # Kiểm tra xem tên cột của bạn là 'image_path' hay 'image'
    img_key = 'image_path' if 'image_path' in example else 'image'
    img = example[img_key]
    ground_truth = example['text'].strip()

    # Tạo file tạm (dùng đường dẫn tuyệt đối)
    temp_path = f"/content/temp_sample_{i}.jpg"

    try:
        # Lưu ảnh từ dataset ra file thực tế
        img.save(temp_path)

        # BẪY HỨNG OUTPUT
        f = io.StringIO()
        with redirect_stdout(f):
            # Gọi model.infer với các tham số chắc chắn không rỗng
            model.infer(
                tokenizer,
                prompt="<image>\nFree OCR. ",
                image_file=temp_path,
                output_path=output_dir,
                base_size=1024,
                image_size=640,
                crop_mode=True,
                save_results=False
            )

        captured_text = f.getvalue()

        # PHÂN TÍCH CHUỖI ĐỂ LẤY KẾT QUẢ (Log của bạn có các dòng gạch ngang)
        # Chúng ta lấy phần nội dung sau cùng, bỏ qua các dòng hệ thống
        lines = [l.strip() for l in captured_text.strip().split('\n') if l.strip()]

        prediction = ""
        # Duyệt ngược từ cuối lên để tìm dòng văn bản OCR đầu tiên gặp phải
        for line in reversed(lines):
            if "====" not in line and "BASE:" not in line and "PATCHES:" not in line:
                prediction = line
                break

        if prediction:
            error = cer(ground_truth, prediction)
            results.append({
                "id": i,
                "ref": ground_truth,
                "pred": prediction,
                "cer": error * 100
            })

    except Exception as e:
        # In lỗi chi tiết để debug
        print(f"\nLỗi ở mẫu {i}: {e}")

    finally:
        # Dọn dẹp file tạm
        if os.path.exists(temp_path):
            os.remove(temp_path)



Bắt đầu đánh giá 201 mẫu...


100%|██████████| 201/201 [16:17<00:00,  4.86s/it]


In [8]:
# --- IN BÁO CÁO Y HỆT MẪU BẠN MUỐN ---
cer_values = [r['cer'] for r in results]

print("\n" + "="*60)
print("Baseline Model Performance (Unsloth DeepSeek-OCR)")
print("="*60)
print(f"Number of samples: {len(results)}")
print(f"Mean CER: {np.mean(cer_values):.2f}%")
print(f"Median CER: {np.median(cer_values):.2f}%")
print(f"Min CER: {np.min(cer_values):.2f}%")
print(f"Max CER: {np.max(cer_values):.2f}%")
print("="*60)

# Sắp xếp để tìm Best/Worst
sorted_res = sorted(results, key=lambda x: x['cer'])

print("\n Best Predictions (Lowest CER):")
for res in sorted_res[:3]:
    print(f"Sample {res['id']} (CER: {res['cer']:.2f}%) | Ref: {res['ref']} | Pred: {res['pred']}")

print("\n Worst Predictions (Highest CER):")
for res in sorted_res[-3:]:
    print(f"Sample {res['id']} (CER: {res['cer']:.2f}%) | Ref: {res['ref']} | Pred: {res['pred']}")


Baseline Model Performance (Unsloth DeepSeek-OCR)
Number of samples: 201
Mean CER: 40.56%
Median CER: 32.31%
Min CER: 6.06%
Max CER: 141.67%

 Best Predictions (Lowest CER):
Sample 103 (CER: 6.06%) | Ref: nghiệp nhà nước và việc tổ chức sắp xếp lại doanh nghiệp nhà nước. | Pred: nghịp nhà nước và việc tổ chức, sắp xếp lời doanh nghiệp nhà nước.
Sample 24 (CER: 7.79%) | Ref: Ngay cả trường hợp ta không qui định, người dân và doanh nghiệp vẫn thực hiện | Pred: Ngay cũ trường hẹp tư liêng qui định, người dân và doanh nghiệp vẫn thực hiện
Sample 38 (CER: 8.06%) | Ref: IX của Đảng bộ TP.HCM Về xây dựng Đảng : tiếp tục chỉ đạo mạnh | Pred: IX câu Đảng bộ TP.HCM về xây dựng Đảng: tiếp tục chỉ đạo mạnh.

 Worst Predictions (Highest CER):
Sample 183 (CER: 100.00%) | Ref: ngờ | Pred: hā
Sample 140 (CER: 114.49%) | Ref: Chúng ta " hoan nghênh và biểu dương những việc làm tốt của công nhân | Pred: chuyện là có hơn nghènh vì nên Chương chuyện nói là là là là là là là là là là là là là là là là là 

In [9]:
import pandas as pd

# --- 3. LƯU KẾT QUẢ ---
if results:
    # Chuyển list các dictionary thành DataFrame
    df = pd.DataFrame(results)

    # Đường dẫn file lưu (đặt trong thư mục kết quả đã tạo)
    csv_filename = os.path.join(output_dir, "ocr_evaluation_results.csv")

    # Lưu file CSV
    # encoding='utf-8-sig' giúp Excel hiển thị đúng tiếng Việt (nếu có)
    # index=False để không lưu cột số thứ tự mặc định của pandas
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    print(f"\nĐã lưu {len(results)} kết quả vào: {csv_filename}")

    # Tính CER trung bình toàn tập
    avg_cer = df['cer'].mean()
    print(f"CER trung bình toàn tập: {avg_cer:.2f}%")
else:
    print("Không có kết quả nào để lưu.")


Đã lưu 201 kết quả vào: /content/drive/MyDrive/result/ocr_evaluation_results.csv
CER trung bình toàn tập: 40.56%


In [11]:
import pandas as pd

# Đường dẫn đến file đã lưu
csv_path = "/content/drive/MyDrive/result/ocr_evaluation_results.csv"

# Đọc file
df_loaded = pd.read_csv(csv_path)

# Hiển thị 5 dòng đầu tiên
print(df_loaded)

# Bạn có thể truy cập lại list results nếu cần:
# loaded_results = df_loaded.to_dict(orient='records')

      id                                                ref  \
0      0  Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùn...   
1      1  cơ chế " khuyến khích nhà đầu tư thỏa thuận vớ...   
2      2  chuyển nhượng hoặc thuê quyền sử dụng đất, nhậ...   
3      3  dụng đất... " Nhưng tại hội nghị hôm qua, tiến...   
4      4  & môi trường lại đều đồng thanh " loại bỏ " đi...   
..   ...                                                ...   
196  196  Sau mỗi mùa thi đại học, có bao " sĩ tử " buồn...   
197  197  cao thật đấy. Nhưng cao mà làm gì khi NV1 lấy ...   
198  198  là thất bại, chỉ là khi thành công - bị - trì ...   
199  199  NV2, NV3. Quan trọng là họ đã nỗ lực hết sức đ...   
200  200  của các kỳ thi, và cũng là bản chất của thành ...   

                                                  pred        cer  
0    Thứ tháng Bảy năm ngày và mới tháng Đáng Hùng ...  45.161290  
1    có thể "khuyên khiếc, mua đánh bỏ thỏa thuận v...  28.767123  
2    chuyên nhúng hoặc thực quyển sử dụ

In [12]:
model = FastVisionModel.get_peft_model(
    model,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model` require gradients


In [13]:
instruction = "<image>\nFree OCR. "

def convert_to_conversation(sample):
    """Convert dataset sample to conversation format"""
    conversation = [
        {
            "role": "<|User|>",
            "content": instruction,
            "images": [sample['image']]
        },
        {
            "role": "<|Assistant|>",
            "content": sample["text"]
        },
    ]
    return {"messages": conversation}



In [14]:
from datasets import Dataset

num_samples = 2000



converted_train = [
    convert_to_conversation(sample)
    for sample in dataset["train"].select(range(2000))
]
converted_valid = [
    convert_to_conversation(sample)
    for sample in dataset["valid"].select(range(50))
]



In [15]:
# @title Create datacollator

import torch
import math
from dataclasses import dataclass
from typing import Dict, List, Any, Tuple
from PIL import Image, ImageOps
from torch.nn.utils.rnn import pad_sequence
import io

from deepseek_ocr.modeling_deepseekocr import (
    format_messages,
    text_encode,
    BasicImageTransform,
    dynamic_preprocess,
)

@dataclass
class DeepSeekOCRDataCollator:
    """
    Args:
        tokenizer: Tokenizer
        model: Model
        image_size: Size for image patches (default: 640)
        base_size: Size for global view (default: 1024)
        crop_mode: Whether to use dynamic cropping for large images
        train_on_responses_only: If True, only train on assistant responses (mask user prompts)
    """
    tokenizer: Any
    model: Any
    image_size: int = 640
    base_size: int = 1024
    crop_mode: bool = True
    image_token_id: int = 128815
    train_on_responses_only: bool = True

    def __init__(
        self,
        tokenizer,
        model,
        image_size: int = 640,
        base_size: int = 1024,
        crop_mode: bool = True,
        train_on_responses_only: bool = True,
    ):
        self.tokenizer = tokenizer
        self.model = model
        self.image_size = image_size
        self.base_size = base_size
        self.crop_mode = crop_mode
        self.image_token_id = 128815
        self.dtype = model.dtype  # Get dtype from model
        self.train_on_responses_only = train_on_responses_only

        self.image_transform = BasicImageTransform(
            mean=(0.5, 0.5, 0.5),
            std=(0.5, 0.5, 0.5),
            normalize=True
        )
        self.patch_size = 16
        self.downsample_ratio = 4

        # Get BOS token ID from tokenizer
        if hasattr(tokenizer, 'bos_token_id') and tokenizer.bos_token_id is not None:
            self.bos_id = tokenizer.bos_token_id
        else:
            self.bos_id = 0
            print(f"Warning: tokenizer has no bos_token_id, using default: {self.bos_id}")

    def deserialize_image(self, image_data) -> Image.Image:
        """Convert image data (bytes dict or PIL Image) to PIL Image in RGB mode"""
        if isinstance(image_data, Image.Image):
            return image_data.convert("RGB")
        elif isinstance(image_data, dict) and 'bytes' in image_data:
            image_bytes = image_data['bytes']
            image = Image.open(io.BytesIO(image_bytes))
            return image.convert("RGB")
        else:
            raise ValueError(f"Unsupported image format: {type(image_data)}")

    def calculate_image_token_count(self, image: Image.Image, crop_ratio: Tuple[int, int]) -> int:
        """Calculate the number of tokens this image will generate"""
        num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
        num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

        width_crop_num, height_crop_num = crop_ratio

        if self.crop_mode:
            img_tokens = num_queries_base * num_queries_base + 1
            if width_crop_num > 1 or height_crop_num > 1:
                img_tokens += (num_queries * width_crop_num + 1) * (num_queries * height_crop_num)
        else:
            img_tokens = num_queries * num_queries + 1

        return img_tokens

    def process_image(self, image: Image.Image) -> Tuple[List, List, List, List, Tuple[int, int]]:
        """
        Process a single image based on crop_mode and size thresholds

        Returns:
            Tuple of (images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio)
        """
        images_list = []
        images_crop_list = []
        images_spatial_crop = []

        if self.crop_mode:
            # Determine crop ratio based on image size
            if image.size[0] <= 640 and image.size[1] <= 640:
                crop_ratio = (1, 1)
                images_crop_raw = []
            else:
                images_crop_raw, crop_ratio = dynamic_preprocess(
                    image, min_num=2, max_num=9,
                    image_size=self.image_size, use_thumbnail=False
                )

            # Process global view with padding
            global_view = ImageOps.pad(
                image, (self.base_size, self.base_size),
                color=tuple(int(x * 255) for x in self.image_transform.mean)
            )
            images_list.append(self.image_transform(global_view).to(self.dtype))

            width_crop_num, height_crop_num = crop_ratio
            images_spatial_crop.append([width_crop_num, height_crop_num])

            # Process local views (crops) if applicable
            if width_crop_num > 1 or height_crop_num > 1:
                for crop_img in images_crop_raw:
                    images_crop_list.append(
                        self.image_transform(crop_img).to(self.dtype)
                    )

            # Calculate image tokens
            num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
            num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

            tokenized_image = ([self.image_token_id] * num_queries_base + [self.image_token_id]) * num_queries_base
            tokenized_image += [self.image_token_id]

            if width_crop_num > 1 or height_crop_num > 1:
                tokenized_image += ([self.image_token_id] * (num_queries * width_crop_num) + [self.image_token_id]) * (
                    num_queries * height_crop_num)

        else:  # crop_mode = False
            crop_ratio = (1, 1)
            images_spatial_crop.append([1, 1])

            # For smaller base sizes, resize; for larger, pad
            if self.base_size <= 640:
                resized_image = image.resize((self.base_size, self.base_size), Image.LANCZOS)
                images_list.append(self.image_transform(resized_image).to(self.dtype))
            else:
                global_view = ImageOps.pad(
                    image, (self.base_size, self.base_size),
                    color=tuple(int(x * 255) for x in self.image_transform.mean)
                )
                images_list.append(self.image_transform(global_view).to(self.dtype))

            num_queries = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)
            tokenized_image = ([self.image_token_id] * num_queries + [self.image_token_id]) * num_queries
            tokenized_image += [self.image_token_id]

        return images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio

    def process_single_sample(self, messages: List[Dict]) -> Dict[str, Any]:
            """
            Process a single conversation into model inputs.
            """

            # --- 1. Setup ---
            images = []
            for message in messages:
                if "images" in message and message["images"]:
                    for img_data in message["images"]:
                        if img_data is not None:
                            pil_image = self.deserialize_image(img_data)
                            images.append(pil_image)

            if not images:
                raise ValueError("No images found in sample. Please ensure all samples contain images.")

            tokenized_str = []
            images_seq_mask = []
            images_list, images_crop_list, images_spatial_crop = [], [], []

            prompt_token_count = -1 # Index to start training
            assistant_started = False
            image_idx = 0

            # Add BOS token at the very beginning
            tokenized_str.append(self.bos_id)
            images_seq_mask.append(False)

            for message in messages:
                role = message["role"]
                content = message["content"]

                # Check if this is the assistant's turn
                if role == "<|Assistant|>":
                    if not assistant_started:
                        # This is the split point. All tokens added *so far*
                        # are part of the prompt.
                        prompt_token_count = len(tokenized_str)
                        assistant_started = True

                    # Append the EOS token string to the *end* of assistant content
                    content = f"{content.strip()} {self.tokenizer.eos_token}"

                # Split this message's content by the image token
                text_splits = content.split('<image>')

                for i, text_sep in enumerate(text_splits):
                    # Tokenize the text part
                    tokenized_sep = text_encode(self.tokenizer, text_sep, bos=False, eos=False)
                    tokenized_str.extend(tokenized_sep)
                    images_seq_mask.extend([False] * len(tokenized_sep))

                    # If this text is followed by an <image> tag
                    if i < len(text_splits) - 1:
                        if image_idx >= len(images):
                            raise ValueError(
                                f"Data mismatch: Found '<image>' token but no corresponding image."
                            )

                        # Process the image
                        image = images[image_idx]
                        img_list, crop_list, spatial_crop, tok_img, _ = self.process_image(image)

                        images_list.extend(img_list)
                        images_crop_list.extend(crop_list)
                        images_spatial_crop.extend(spatial_crop)

                        # Add image placeholder tokens
                        tokenized_str.extend(tok_img)
                        images_seq_mask.extend([True] * len(tok_img))

                        image_idx += 1 # Move to the next image

            # --- 3. Validation and Final Prep ---
            if image_idx != len(images):
                raise ValueError(
                    f"Data mismatch: Found {len(images)} images but only {image_idx} '<image>' tokens were used."
                )

            # If we never found an assistant message, we're in a weird state
            # (e.g., user-only prompt). We mask everything.
            if not assistant_started:
                print("Warning: No assistant message found in sample. Masking all tokens.")
                prompt_token_count = len(tokenized_str)

            # Prepare image tensors
            images_ori = torch.stack(images_list, dim=0)
            images_spatial_crop_tensor = torch.tensor(images_spatial_crop, dtype=torch.long)

            if images_crop_list:
                images_crop = torch.stack(images_crop_list, dim=0)
            else:
                images_crop = torch.zeros((1, 3, self.base_size, self.base_size), dtype=self.dtype)

            return {
                "input_ids": torch.tensor(tokenized_str, dtype=torch.long),
                "images_seq_mask": torch.tensor(images_seq_mask, dtype=torch.bool),
                "images_ori": images_ori,
                "images_crop": images_crop,
                "images_spatial_crop": images_spatial_crop_tensor,
                "prompt_token_count": prompt_token_count, # This is now accurate
            }

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        """Collate batch of samples"""
        batch_data = []

        # Process each sample
        for feature in features:
            try:
                processed = self.process_single_sample(feature['messages'])
                batch_data.append(processed)
            except Exception as e:
                print(f"Error processing sample: {e}")
                continue

        if not batch_data:
            raise ValueError("No valid samples in batch")

        # Extract lists
        input_ids_list = [item['input_ids'] for item in batch_data]
        images_seq_mask_list = [item['images_seq_mask'] for item in batch_data]
        prompt_token_counts = [item['prompt_token_count'] for item in batch_data]

        # Pad sequences
        input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        images_seq_mask = pad_sequence(images_seq_mask_list, batch_first=True, padding_value=False)

        # Create labels
        labels = input_ids.clone()

        # Mask padding tokens
        labels[labels == self.tokenizer.pad_token_id] = -100

        # Mask image tokens (model shouldn't predict these)
        labels[images_seq_mask] = -100

        # Mask user prompt tokens when train_on_responses_only=True (only train on assistant responses)
        if self.train_on_responses_only:
            for idx, prompt_count in enumerate(prompt_token_counts):
                if prompt_count > 0:
                    labels[idx, :prompt_count] = -100

        # Create attention mask
        attention_mask = (input_ids != self.tokenizer.pad_token_id).long()

        # Prepare images batch (list of tuples)
        images_batch = []
        for item in batch_data:
            images_batch.append((item['images_crop'], item['images_ori']))

        # Stack spatial crop info
        images_spatial_crop = torch.cat([item['images_spatial_crop'] for item in batch_data], dim=0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "images": images_batch,
            "images_seq_mask": images_seq_mask,
            "images_spatial_crop": images_spatial_crop,
        }

In [16]:
from transformers import Trainer, TrainingArguments
from unsloth import is_bf16_supported
FastVisionModel.for_training(model) # Enable for training!
data_collator = DeepSeekOCRDataCollator(
    tokenizer=tokenizer,
    model = model,
    image_size=640,
    base_size=1024,
    crop_mode=True,
    train_on_responses_only=True,
)


trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = data_collator, # Must use!
    train_dataset = converted_train,
    eval_dataset = converted_valid,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = -1, # Để -1 để nó ưu tiên chạy theo epoch
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        fp16 = not is_bf16_supported(),  # Use fp16 if bf16 is not supported
        bf16 = is_bf16_supported(),  # Use bf16 if supported

        report_to = "none",     # For Weights and Biases
        dataloader_num_workers=2,
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
            # --- CÁC DÒNG QUAN TRỌNG CHO VALIDATION ---
        eval_strategy = "steps",        # Bật chế độ đánh giá theo bước
        eval_steps = 100,                # Cứ 10 bước train thì chạy Val một lần
        per_device_eval_batch_size = 2, # Batch size riêng cho lúc Val
        save_strategy = "steps",
        save_steps = 50,
        output_dir = "/content/drive/MyDrive/outputs", # Trỏ vào Drive của bạn
        # Nếu trong Drive đã có thư mục checkpoint

        # ------------------------------------------
    ),
)

/tmp/ipython-input-3242350462.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 77,509,632 of 3,413,615,872 (2.27% trained)
	save_steps: 50 (from args) != 100 (from trainer_state.json)


Step,Training Loss,Validation Loss


In [19]:
import io
import numpy as np
import os
from contextlib import redirect_stdout
from jiwer import cer
from tqdm import tqdm

# --- 1. KIỂM TRA VÀ TẠO THƯ MỤC KẾT QUẢ ---
output_dir = "/content/drive/MyDrive/result_finetuned" # Dùng đường dẫn tuyệt đối cho chắc chắn
os.makedirs(output_dir, exist_ok=True)

# --- 2. CẤU HÌNH TẬP TEST ---
num_samples = 201
# Đảm bảo bạn đang dùng đúng split 'test'
test_subset = dataset['test'].select(range(min(num_samples, len(dataset['test']))))
results = []

print(f"Bắt đầu đánh giá {len(test_subset)} mẫu...")

for i, example in enumerate(tqdm(test_subset)):
    # Kiểm tra xem tên cột của bạn là 'image_path' hay 'image'
    img_key = 'image_path' if 'image_path' in example else 'image'
    img = example[img_key]
    ground_truth = example['text'].strip()

    # Tạo file tạm (dùng đường dẫn tuyệt đối)
    temp_path = f"/content/temp_sample_{i}.jpg"

    try:
        # Lưu ảnh từ dataset ra file thực tế
        img.save(temp_path)

        # BẪY HỨNG OUTPUT
        f = io.StringIO()
        with redirect_stdout(f):
            # Gọi model.infer với các tham số chắc chắn không rỗng
            model.infer(
                tokenizer,
                prompt="<image>\nFree OCR. ",
                image_file=temp_path,
                output_path=output_dir,
                base_size=1024,
                image_size=640,
                crop_mode=True,
                save_results=False
            )

        captured_text = f.getvalue()

        # PHÂN TÍCH CHUỖI ĐỂ LẤY KẾT QUẢ (Log của bạn có các dòng gạch ngang)
        # Chúng ta lấy phần nội dung sau cùng, bỏ qua các dòng hệ thống
        lines = [l.strip() for l in captured_text.strip().split('\n') if l.strip()]

        prediction = ""
        # Duyệt ngược từ cuối lên để tìm dòng văn bản OCR đầu tiên gặp phải
        for line in reversed(lines):
            if "====" not in line and "BASE:" not in line and "PATCHES:" not in line:
                prediction = line
                break

        if prediction:
            error = cer(ground_truth, prediction)
            results.append({
                "id": i,
                "ref": ground_truth,
                "pred": prediction,
                "cer": error * 100
            })

    except Exception as e:
        # In lỗi chi tiết để debug
        print(f"\nLỗi ở mẫu {i}: {e}")

    finally:
        # Dọn dẹp file tạm
        if os.path.exists(temp_path):
            os.remove(temp_path)



Bắt đầu đánh giá 201 mẫu...


100%|██████████| 201/201 [25:02<00:00,  7.48s/it]


In [20]:
# --- IN BÁO CÁO Y HỆT MẪU BẠN MUỐN ---
cer_values = [r['cer'] for r in results]

print("\n" + "="*60)
print("Baseline Model Performance (Unsloth DeepSeek-OCR)")
print("="*60)
print(f"Number of samples: {len(results)}")
print(f"Mean CER: {np.mean(cer_values):.2f}%")
print(f"Median CER: {np.median(cer_values):.2f}%")
print(f"Min CER: {np.min(cer_values):.2f}%")
print(f"Max CER: {np.max(cer_values):.2f}%")
print("="*60)

# Sắp xếp để tìm Best/Worst
sorted_res = sorted(results, key=lambda x: x['cer'])

print("\n Best Predictions (Lowest CER):")
for res in sorted_res[:3]:
    print(f"Sample {res['id']} (CER: {res['cer']:.2f}%) | Ref: {res['ref']} | Pred: {res['pred']}")

print("\n Worst Predictions (Highest CER):")
for res in sorted_res[-3:]:
    print(f"Sample {res['id']} (CER: {res['cer']:.2f}%) | Ref: {res['ref']} | Pred: {res['pred']}")


Baseline Model Performance (Unsloth DeepSeek-OCR)
Number of samples: 201
Mean CER: 17.25%
Median CER: 12.31%
Min CER: 0.00%
Max CER: 76.92%

 Best Predictions (Lowest CER):
Sample 37 (CER: 0.00%) | Ref: Chương trình hành động thực hiện Nghị quyết trung ương 9 - khóa | Pred: Chương trình hành động thực hiện Nghị quyết trung ương 9 - khóa
Sample 67 (CER: 0.00%) | Ref: doanh nghiệp nhà nước và việc tổ chức sắp xếp lại doanh nghiệp nhà | Pred: doanh nghiệp nhà nước và việc tổ chức sắp xếp lại doanh nghiệp nhà
Sample 69 (CER: 0.00%) | Ref: đối với các dự án có vốn đầu tư nước ngoài. Rà soát lại chương | Pred: đối với các dự án có vốn đầu tư nước ngoài. Rà soát lại chương

 Worst Predictions (Highest CER):
Sample 152 (CER: 60.94%) | Ref: đã làm điều đó? Ba năm trôi qua, chắc hẳn đã hàng trăm, ngàn lần | Pred: để bán được dịch Bạc năm tận nạn, chết hầu đã hãy tiêm ngừa, người họ
Sample 145 (CER: 71.01%) | Ref: những sai phạm đó. Có như vậy niềm tin vào lẽ phải mới được khôi phục | Pred: chươ

In [21]:
import pandas as pd

# --- 3. LƯU KẾT QUẢ ---
if results:
    # Chuyển list các dictionary thành DataFrame
    df = pd.DataFrame(results)

    # Đường dẫn file lưu (đặt trong thư mục kết quả đã tạo)
    csv_filename = os.path.join(output_dir, "ocr_evaluation_results_finetuned.csv")

    # Lưu file CSV
    # encoding='utf-8-sig' giúp Excel hiển thị đúng tiếng Việt (nếu có)
    # index=False để không lưu cột số thứ tự mặc định của pandas
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

    print(f"\nĐã lưu {len(results)} kết quả vào: {csv_filename}")

    # Tính CER trung bình toàn tập
    avg_cer = df['cer'].mean()
    print(f"CER trung bình toàn tập: {avg_cer:.2f}%")
else:
    print("Không có kết quả nào để lưu.")


Đã lưu 201 kết quả vào: /content/drive/MyDrive/result_finetuned/ocr_evaluation_results_finetuned.csv
CER trung bình toàn tập: 17.25%


In [22]:
import pandas as pd

# Đường dẫn đến file đã lưu
csv_path = "/content/drive/MyDrive/result_finetuned/ocr_evaluation_results_finetuned.csv"

# Đọc file
df_loaded = pd.read_csv(csv_path)

# Hiển thị 5 dòng đầu tiên
print(df_loaded.head())

# Bạn có thể truy cập lại list results nếu cần:
# loaded_results = df_loaded.to_dict(orient='records')

   id                                                ref  \
0   0  Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùn...   
1   1  cơ chế " khuyến khích nhà đầu tư thỏa thuận vớ...   
2   2  chuyển nhượng hoặc thuê quyền sử dụng đất, nhậ...   
3   3  dụng đất... " Nhưng tại hội nghị hôm qua, tiến...   
4   4  & môi trường lại đều đồng thanh " loại bỏ " đi...   

                                                pred        cer  
0  Thứ trưởng Bộ Tài nguyên và môi trường Đảng Hư...  14.516129  
1  có chế " khuyến khích, mà đầu tư thỏa thuận vớ...  20.547945  
2  chuyển những hoặc thực quyết sử dụng đất, nhằm...  15.942029  
3  dụng đất... " Nhưng tại hội nghị hôm qua, tiến...   2.777778  
4  và mới trường lên tiến hành " toạ bộ " điều kh...  22.666667  
